In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import (
    ArrayType,
    DateType,
    DecimalType,
    IntegerType,
    LongType,
    StringType,
    StructField,
    StructType,
)

In [5]:
spark: SparkSession = (
    SparkSession.builder.appName("StockStream").master("local")
    .config(
        "spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0,com.amazonaws:aws-java-sdk:1.12.183,org.apache.hadoop:hadoop-aws:3.2.2",
    )
    .config("spark.sql.shuffle.partitions", "4").getOrCreate()
)

In [28]:
kafka_df = (
    spark.read.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "stock-data")
    .option("startingOffsets", "earliest")
    .option("group.id", "stock-stream-consumer")
    .option("failOnDataLoss", False)
    .load()
)

In [29]:
kafka_df.show()

+-------------+--------------------+----------+---------+------+--------------------+-------------+
|          key|               value|     topic|partition|offset|           timestamp|timestampType|
+-------------+--------------------+----------+---------+------+--------------------+-------------+
|[41 41 50 4C]|[7B 22 63 6F 6D 7...|stock-data|        0|     0|2024-12-29 11:37:...|            0|
|[41 41 50 4C]|[7B 22 63 6F 6D 7...|stock-data|        0|     1|2024-12-29 11:38:...|            0|
|         NULL|[7B 22 74 79 70 6...|stock-data|        1|     0|2024-12-28 23:50:...|            0|
|         NULL|[7B 22 74 79 70 6...|stock-data|        1|     1|2024-12-28 23:50:...|            0|
|         NULL|[7B 22 74 79 70 6...|stock-data|        2|     0|2024-12-28 23:50:...|            0|
+-------------+--------------------+----------+---------+------+--------------------+-------------+



In [30]:
# kafka_df.writeStream.format("console").outputMode("append").start().awaitTermination()

In [31]:
schema = StructType(
    [
        StructField("type", StringType()),
        StructField(
            "data",
            ArrayType(
                StructType(
                    [
                        StructField("p", DecimalType()),
                        StructField("s", StringType()),
                        StructField("t", LongType()),
                        StructField("v", DecimalType()),
                        StructField("c", IntegerType()),
                    ]
                )
            ),
        ),
        StructField(
            "companyProfile",
            StructType(
                [
                    StructField("country", StringType()),
                    StructField("currency", StringType()),
                    StructField("exchange", StringType()),
                    StructField("ipo", DateType()),
                    StructField("marketCapitalization", DecimalType()),
                    StructField("name", StringType()),
                    StructField("phone", StringType()),
                    StructField("shareOutstanding", DecimalType()),
                    StructField("ticker", StringType()),
                    StructField("weburl", StringType()),
                    StructField("logo", StringType()),
                    StructField("finnhubIndustry", StringType()),
                ]
            ),
        ),
    ]
)

In [32]:
kafka_df = kafka_df\
    .withColumn("key", f.col("key").cast(StringType()))\
    .withColumn("value", f.col("value").cast(StringType()))

In [33]:
raw_df = kafka_df.withColumn("symbol",f.col("key")).withColumn("value",f.from_json(f.col("value"),schema)).select("symbol","value.*")

In [34]:
raw_df.show()

+------+-----+--------------------+--------------------+
|symbol| type|                data|      companyProfile|
+------+-----+--------------------+--------------------+
|  AAPL|trade|[{256, AAPL, 1735...|{US, USD, NASDAQ/...|
|  AAPL|trade|[{256, AAPL, 1735...|{US, USD, NASDAQ/...|
|  NULL| ping|                NULL|                NULL|
|  NULL| ping|                NULL|                NULL|
|  NULL| ping|                NULL|                NULL|
+------+-----+--------------------+--------------------+



In [35]:
raw_df = raw_df.filter(f.col("symbol").isNotNull())

In [36]:
raw_df.show()

+------+-----+--------------------+--------------------+
|symbol| type|                data|      companyProfile|
+------+-----+--------------------+--------------------+
|  AAPL|trade|[{256, AAPL, 1735...|{US, USD, NASDAQ/...|
|  AAPL|trade|[{256, AAPL, 1735...|{US, USD, NASDAQ/...|
+------+-----+--------------------+--------------------+



In [37]:
company_data = raw_df.select("companyProfile.*")

In [57]:
price_data = raw_df.select("symbol",f.posexplode("data").alias("pos","data")).drop("pos")

In [58]:
price_data.show()

+------+--------------------+
|symbol|                data|
+------+--------------------+
|  AAPL|{256, AAPL, 17354...|
|  AAPL|{256, AAPL, 17354...|
+------+--------------------+



In [59]:
price_data = price_data.select(f.col("symbol").alias("ticker"),"data.*")

In [60]:
price_data.show()

+------+---+----+-------------+----+----+
|ticker|  p|   s|            t|   v|   c|
+------+---+----+-------------+----+----+
|  AAPL|256|AAPL|1735451912469|1300|NULL|
|  AAPL|256|AAPL|1735451912469|1300|NULL|
+------+---+----+-------------+----+----+



In [ ]:
final_price_data = 